![First Power Bi](.\images\dbachecks-logo.png )
# dbachecks

## Installation 

Take a look at the installation guidelines here [Intro](./01-dbachecks-Intro.ipyLets set some variaailable

In [ ]:
$FolderPath = $Env:USERPROFILE + '\Documents\dbachecks'
$SqlInstances = $Sql1,$SQL2 = 'localhost,15592','localhost,15593'
$SqlCredential = Import-Clixml -Path $FolderPath\sqladmin.cred

Lets Check if you have the dbachecks module available

In [ ]:
Get-Module dbachecks -ListAvailable

## Examine the available commands

You can look at the commands in the dbachecks module

In [ ]:
Get-Command -Module dbachecks

# All of the checks

You can see all of the checks using `Get-DbcCheck` 


In [ ]:
Get-DbcCheck

## Searching for Checks to run

You can search for checks using `Get-DbcCheck -Pattern SEARCHTERM`

In [ ]:
Get-DbcCheck -Pattern backup

# Running a Check

This requires that you have the containers set up from the Notebook 00 - Setting up the containers for the rest of the containers.ipynb

If you have altered the folder path you will need to alter it below.

Running a check from the command line can be done with

`Invoke-DbcCheck -SqlInstance SQLINSTANCE -Check UNIQUETAG, A TAG, GROUP`

You can run multiple checks with

`Invoke-DbcCheck -SqlInstance SQLINSTANCE -Check Tag,AnotherTag,AnotherTag`

In [ ]:
#$null = Reset-DbcConfig
Invoke-DbcCheck -SqlInstance $SQL1 -SqlCredential $SqlCredential -Check AutoClose

## Auto-Complete for Checks
At the command line and in the editor the Checks will auto-complete

Place the cursor a space after the `-Check` and press CTRL + SPACE

`Invoke-DbcCheck -SqlInstance SQL2016N1 -Check `

In [ ]:

Invoke-DbcCheck -SqlInstance $SQL1 -SqlCredential $SqlCredential -Check LogfileSize

# Check Agent Jobs at the command line

Did all of the Agent Jobs succeed ?

In [ ]:

Invoke-DbcCheck -SqlInstance $SQL1 -SqlCredential $SqlCredential -Check FailedJob

You may have no results because they are no enabled jobs that have run in the last 7 days (depending upon when I last updated the image). Lets run the jobs and see what we get

In [ ]:
(Get-DbaAgentJob -SqlInstance $SQL1 -SqlCredential $SqlCredential).Start()
Start-Sleep -Seconds 15
Invoke-DbcCheck -SqlInstance $SQL1 -SqlCredential $SqlCredential -Check FailedJob

## Check the instances are available
Are all of the instances up and available?

There will be failures for the default values of the authentication scheme as we are connecting with SQL Authentication. We will look at configuration in a later notebook.

In [ ]:
Invoke-DbcCheck -SqlInstance $SqlInstances -SqlCredential $SqlCredential -Check InstanceConnection 

# Check the databases are available
Are all the databases up available?

This by default will only show the system databases but we will talk about configuration for this in the next notebook

In [ ]:
Invoke-DbcCheck -SqlInstance $SqlInstances -SqlCredential $SqlCredential  -Check DatabaseStatus

## Check that DBCC CheckDB 

This is one of my favourite checks to run. You can get the last time DBCC CHECKDB was run for a single database using T-SQL with

```sql
DBCC DBINFO('DatabaseName') WITH TABLERESULTS
```

You will find the result somewhere around line 50. Doing that for all of the databases on your estate is time consuming and prone to (human) error.
With dbachecks you can do it like this. ON my machine this checks the 2 containers and 14 databases to make sure DBCC CHECKDB was run in the last 7 days in under 2 seconds :-)

In [ ]:
Invoke-DbcCheck -SqlInstance $SqlInstances -SqlCredential $SqlCredential  -Check LastGoodCheckDb

## Check that the Error log has no Errors

By default this will check the error logs for the last 24 hours

In [ ]:
Invoke-DbcCheck -SqlInstance $SqlInstances -SqlCredential $SqlCredential  -Check ErrorLog

## Check the disk space on the instance

This will check that the disk space on the hosts has at least 20% free space by default. It requires passing the host names to the `-ComputerName` parameter

In [ ]:
Invoke-DbcCheck -ComputerName $SqlInstances -Check DiskCapacity

# Check if DAC is enabled

This will check that DAC - the dedicated administrator connection is enabled (by default)

In [ ]:
Invoke-DbcCheck -SqlInstance $SqlInstances -SqlCredential $SqlCredential  -Check DAC

## Check if the database collation matches the server collation

This will check that the collation of the database matches the collation of the instance to avoid collation conflict errors when using temporary tables

In [ ]:
Invoke-DbcCheck -SqlInstance $SqlInstances -SqlCredential $SqlCredential  -Check DatabaseCollation

Are all of the SPNs correct ?

You should not run this one but look at the saved results to see what you can get.

In [ ]:
$Instances =  'SQL2005Ser2003', 'SQL2008Ser12R2', 'SQL2012Ser08AG1', 'SQL2012Ser08AG2', 'SQL2012Ser08AG3', 'SQL2014Ser12R2', 'SQL2016N1', 'SQL2016N2', 'SQL2016N3', 'SQL2017N5', 'SQL2019N20', 'SQL2019N21', 'SQL2019N22', 'SQL2019N5'
Invoke-DbcCheck -SqlInstance $instances -Check SPN

# Running multiple checks

All of the examples above have used a single unique tag. You can run more than one check at a time. You can do this either by specifying a tag that covers a number of checks. You can find these by running `Get-DbcCheck`

When you search for checks like disk

In [ ]:
Get-DbcCheck disk

you can see that under AllTags for the DiskCapacity check there is a tag called Storage. If you search for storage

In [ ]:
Get-DbcCheck Storage

you can see that this returns two checks - BackUpPathAccess and DiskCapacity. You can run both of these checks at the same time. Note that the `Type` of check is different for these two checks and you will have to supply both `SqlInstance` and `ComputerName`

In [ ]:
Invoke-DbcCheck -SqlInstance $SqlInstances -ComputerName $SqlInstances -SqlCredential $SqlCredential -Check Storage

## Running checks for a Group

Checks are grouped into ... well groups! The groups are Database, Instance, Server, Agent, HADR, LogShipping and MaintenanceSolution. You can run all of the checks in a group by specifying the group. You will see failures and some errors due to missing configurations and because some checks wont run against containers. I think it is worth showing hte test failures so that you can see what they look like.  

This takes about 90 seconds on my machine

In [ ]:
Invoke-DbcCheck -SqlInstance $SqlInstances -SqlCredential $SqlCredential -Check Instance